# <p style="text-align:center" color='red' ><font color='blue'> Edge-assisted DASH Video Caching in 5G Networks </font> </p>


# Packages

In [56]:
import numpy as np
import matplotlib.pyplot as plt
import random
import itertools
import math
from scipy.spatial import distance
from gurobipy import *
import networkx as nx 
import time


In [57]:
objectiveName = 'Obj-cacheHit'
#objectiveName = 'Obj-byteHit'

# Data Units

In [58]:
Byteps = 8.0#bit
Kbps = KB = 1000.0
Mbps = MB = 1000.0 * KB
Gbps = GB = 1000.0 * MB
GHz = 1000000000.0

In [59]:
# A set of 3 gNodeBs are connected to each other
# through 20 Gbps Xn backhaul links, while each of the three
# gNodeBs is connected to the aggregation point using 20 Gbps
# N G backhaul links which in turn is connected to the 5G
# core network using 50 Gbps backhaul links

# <p style="text-align:center" color='red' ><font color='red'>  Substrate Network </font> </p>


# Number of resources 

In [60]:
# Parameters NUMBERS
n_gnb = 3 #Number of gNBs nodes (Each gNB is collocated with one MEC node)
n_core = 1 #Number of core nodes
n = n_gnb + n_core  #Number of nodes in the network (the sumation of gNBs + and core server)

n_vids = 2 # Number of videos
n_seg = 2 # number of video segments (Videos can be divided into segments with equal duration)
n_qul = 5 # Each segment is available in multiple qualities

segDuration = 8 #second


nodeStorCap = 1 * GB
x2LinkCap = 20 * Gbps
backhaulLinkCap = 2 * Gbps


# Sets

In [61]:
N_gnb  = {'Node'+str(n) for n in range (1, n_gnb + 1)} #Set of gNBs
N_core = {'Node'+str(n) for n in range (0, n_core)}  # Set of Video Servers
N = N_gnb | N_core # set of all nodes

G = nx.DiGraph(name = 'MNO architectue')  # Network graph
G.clear()

G.add_nodes_from(N_core, numCores = 4,CPU = 2.4 )   # Adding nodes to the graph G
G.add_nodes_from(N_gnb,  numCores = 4,CPU = 1.6)

for i in N_gnb:   # seting storage attribiute for all the nodes
    G.nodes[i]['storage'] = nodeStorCap

G.add_edges_from((i,j, {'capacity': backhaulLinkCap}) for i in N_core for j in N_gnb) # Adding links from core to the gNBs to the graph G
G.add_edges_from((j,i, {'capacity': backhaulLinkCap}) for i in N_core for j in N_gnb) # Adding links from core to the gNBs to the graph G
G.add_edges_from((i,j, {'capacity': x2LinkCap}) for i in N_gnb for j in N_gnb if abs(int(i[4]) - int(j[4])) == 1) # Adding the link between neighbor gNodeBs to the graph G

N_vids = {'Vid' + str(i) for i in range (1, n_vids+1)} # Set of videos
N_seg = {'Seg' + str(i) for i in range (1, n_seg+1)} # Set of segments
N_qul = {'Qual' + str(i) for i in range (1, n_qul+1)} # Set of qualities


# Bitrate is the amount of data encoded for a unit of time, and for streaming is usually referenced in megabits
#per second (Mbps) for video, and in kilobits per second (kbps) for audio. From a streaming perspective, a higher
#video bitrate means a higher quality video that requires more bandwidth.

# Recommended video bitrates for SDR uploads   (https://support.google.com/youtube/answer/1722171?hl=en)
# Type            Video Bitrate (Standard Frame Rate) (24, 25, 30)       QualityClass    

# 2160p (4k)       35-45 Mbps                                                   Q6
# 1440p (2k)       16 Mbps                                                      Q5
# 1080p            8 Mbps                                                       Q4
# 720p             5 Mbps                                                       Q3
# 480p             2.5 Mbps                                                     Q2
# 360p             1 Mbps                                                       Q1

#Qualities
#1080p Q5
#720p Q4
#480p Q3
#360p Q2
#2409 Q1

quality, bitrate = multidict({
    'Qual5': 5 * Mbps,
    'Qual4': 3 * Mbps, 
    'Qual3': 1.5 * Mbps, 
    'Qual2': 0.8 * Mbps,
    'Qual1': 0.4 * Mbps,
    
})


quality, maxTransQual = multidict ({
    ('Qual5', 'Qual4'): 2,
    ('Qual5', 'Qual3'): 3, 
    ('Qual5', 'Qual2'): 4, 
    ('Qual5', 'Qual1'): 6,
    ('Qual4', 'Qual3'): 4,
    ('Qual4', 'Qual2'): 6,
    ('Qual4', 'Qual1'): 9, 
    ('Qual3', 'Qual2'): 8, 
    ('Qual3', 'Qual1'): 11,
    })
 

user, bsAssociation = multidict({ # a dictionary for the size of a video segment in a specific quality 
    'UE1' : ['Node2'],
    'UE2' : ['Node2'], 
})

# Virtual Request Set

In [62]:
n_ue = 2  #Number of users
N_ue = {'UE' + str(i) for i in range (1, n_ue + 1)}

In [63]:
user, userBitrate, userVidSegQaul = multidict({ # a dictionary for the size of a video segment in a specific quality 
    'UE1' : [10 * Mbps, ['Vid1', 'Seg1', 'Qual2']],
    'UE2' : [20 * Mbps, ['Vid2', 'Seg1', 'Qual2']], 
})

In [64]:
vsqnuCombinition = [(v, s, h, n ,u) for v in N_vids for s in N_seg for h in N_qul for n in N_gnb for u in N_ue] # all possible cases for all the videos, segments, and qualities
vsqnu, weight = multidict({ # a dictionary to sotre the weight (priority) of each video, segment, quality, node, user combanition 
    item: 1 for item in vsqnuCombinition
})

# Model

In [65]:
model=Model("Caching")
model.reset(0)

# Variables

$\boldsymbol{\chi_{n}^{v, s, q}}$  |  Indicates if the quality $q \in N_{qul}^{v, s}$ of the segment $s \in N_{seg}^{v}$ of video $v \in N_{vids}$ has been mapped on the edge node $n \in N_{gnb}$.

$\boldsymbol{\chi_{n, u}^{v, s, q}}$  |  Indicates if the quality $q \in N_{qul}^{v, s}$ of the segment $s \in N_{seg}^{v}$ of video $v \in N_{vids}$ has been mapped on the node $n \in N$ and it is assigned to the UE $u \in N_{ue}$.

$\boldsymbol{\chi_{e}^{ \bar{e}}}$  |  Indicates if the virtual link $\bar{e} \in \bar{E}$ is mapped on the substrate link $e \in E$.

In [66]:
X_vsq_n = {}
for v in N_vids:
    for s in N_seg:
        for q in N_qul:
            for n in N:
                X_vsq_n [v, s, q, n] = model.addVar(vtype = GRB.BINARY, name = 'X_vsq_n [%s, %s, %s, %s]' % (v, s, q, n))


In [67]:
X_vsq_nu = {}
for n in N:
    for u in N_ue:
        for h in N_qul:
            #if h >= userVidSegQaul[u][2]:
                X_vsq_nu [userVidSegQaul[u][0],userVidSegQaul[u][1], h, n, u] = \
                model.addVar(vtype = GRB.BINARY, name = 'X_vsq_nu [%s, %s, %s, %s, %s]' % (userVidSegQaul[u][0], \
                userVidSegQaul[u][1],h, n, u))


In [68]:
X_ebar_e = {} 
for u in N_ue:
    for e in list(G.edges):
            X_ebar_e[u, e] = model.addVar(vtype = GRB.BINARY, name = 'X_ebar_e [%s, %s]' % (u, e))

                

# Objective Functions

\begin{equation}\label{eq:1}
\boldsymbol{CacheHit:} \quad Max {\sum_{u \in N_{ue}} \sum _{n \in N_{gnb}} \sum _{\substack{h \in N_{qul}^{v, s} \\ h \geq q}} \alpha_{n} \chi_{n, h}^{u, v, s, q}}
\end{equation}


In [69]:
#CORRECT

# this line enforces the model to mapp all the video segment qualities at the core
# the number of video segment quality on the nodes should be minimized to have better utilization of the cache storage
# mapping on the links should be minimized to avoid of selecting links without being necessery

if objectiveName == 'Obj-cacheHit':
    model.setObjective(quicksum(10000 * weight[userVidSegQaul[u][0], userVidSegQaul[u][1], h, n, u] \
                                * X_vsq_nu[userVidSegQaul[u][0],userVidSegQaul[u][1], h, n, u]\
                                for u in N_ue for n in N_gnb for h in N_qul if h >= userVidSegQaul[u][2]) +
                               quicksum(X_vsq_n[v, s, q, n] for v in N_vids for s in N_seg for q in N_qul for n in N_core)\
                            - quicksum(X_vsq_n[v, s, q, n] for v in N_vids for s in N_seg for q in N_qul for n in N_gnb)\
                            - quicksum(X_ebar_e[u, e] for u in N_ue for e in list(G.edges)), GRB.MAXIMIZE) 

Since the size of video content is usually several orders of magnitude larger than web objects, a cache may quickly run out of storage.
Consequently, the temporal locality of video content cannot be well exploited. This will result in a lower byte-hit ratio,
which is defined as the amount of the requested data found in the cache divided by the total amount of the requested
data.

\begin{equation}\label{eq:1}
\boldsymbol{ByteHit:} \quad Max {\sum_{u \in N_{ue}} \sum _{n \in N_{gnb}} \sum _{\substack{h \in N_{qul}^{v, s} \\ h \geq q}} \alpha_{n} \chi_{n, h}^{u, v, s, q} \omega^{v, s, q} \tau^s}
\end{equation}

In [70]:
#CORRECT

# this line enforces the model to mapp all the video segment qualities at the core
# the number of video segment quality on the nodes should be minimized to have better utilization of the cache storage
# mapping on the links should be minimized to avoid of selecting links without being necessery

if objectiveName == 'Obj-byteHit':
    model.setObjective(quicksum(10000 * weight[userVidSegQaul[u][0], userVidSegQaul[u][1], h, n, u] \
                                * size[X_vsq_nu[userVidSegQaul[u][0],userVidSegQaul[u][1], h]]* X_vsq_nu[userVidSegQaul[u][0],userVidSegQaul[u][1], h, n, u]\
                                for u in N_ue for n in N_gnb for h in N_qul if h >= userVidSegQaul[u][2]) +
                               quicksum(X_vsq_n[v, s, q, n] for v in N_vids for s in N_seg for q in N_qul for n in N_core)\
                            - quicksum(X_vsq_n[v, s, q, n] for v in N_vids for s in N_seg for q in N_qul for n in N_gnb)\
                            - quicksum(X_ebar_e[u, e] for u in N_ue for e in list(G.edges)), GRB.MAXIMIZE) 

# Costraints

The first constraint ensures that the storage capacity of the edge nodes are respected and the amount of storage used for storing the videos is less than or equal to the maximum storage capacity of the nodes.

\begin{equation}\label{eq:3}
 {\forall n \in N_{gnb}: \sum_{v \in N_{vids}^{u}} \sum_{s \in N_{seg}^{u, v}} \sum_{q \in N_{qul}^{u, v, s}} \chi_{n}^{v, s, q} \omega^{v, s, q} \tau^s \leq \mathcal{C}_{stor}(n)}
\end{equation}

In [71]:
#CORECT
for n in N_gnb:
    model.addConstr(quicksum(bitrate[q]*segDuration * X_vsq_n[v, s, q, n] for v in N_vids for s in N_seg for q in N_qul) \
                             <= G.nodes[n]['storage'] , name="C3")

At any given time, each user should be provided with one video representation (video, segment, quality) that is requesting.

\begin{equation}
{\forall u \in \bar{N}_{ue} \sum_{n \in N} \sum _{\substack{h \in N_{qul}^{v, s} \\ h \geq q}}:  \chi_{n, u}^{v, s, h} = 1}
\end{equation}



In [72]:
#CORRECT
for u in N_ue:
       model.addConstr(quicksum(X_vsq_nu[userVidSegQaul[u][0],userVidSegQaul[u][1], h, n, u] \
                    for n in G.nodes for h in N_qul if h >= userVidSegQaul[u][2] ) <= 1, name="C4")

The following constraint guarantees that a video is feteched to the edge only if at least one user hase been serving from the video chunk or at least one user is requesting the video chunck.

\begin{equation}\label{eq:7}
{\forall n \in N, \forall v \in N_{vids}^{u} \forall s \in N_{seg}^{u} \forall h \in N_{qual} h>=q: \sum_{u \in \bar{N}_{ue}}\chi^{v, s, h}_{u, n} - \mu* \chi^{v, s, h}_{n} \leq 0}
\end{equation}

In [73]:
M =1000 
for n in N:
    for v in N_vids:
        for s in N_seg:
            for h in N_qul:
                model.addConstr(quicksum(X_vsq_nu[v, s, h, n, u] - M * X_vsq_n[v, s, h, n] \
                for u in N_ue if v == userVidSegQaul[u][0] if s == userVidSegQaul[u][1]) <= 0, name = "CX")

this Constraint ensures that the virtual links can be mapped onto a substrate link as long as the link has sufficient capacity:
\begin{equation}\label{eq:5}
 { \forall e \in E: \sum_{\bar{e} \in \bar{E}}\chi_{e}^{ \bar{e}}  \omega^{\bar{e}}_{bwt}  \leq \mathcal{C}^{e}_{bwt}}
\end{equation}


In [74]:
for e in G.edges:
    model.addConstr(quicksum(X_ebar_e[u, e] * userBitrate[u] for u in N_ue) \
                    <=  G.edges[(e)]['capacity'], name = "C5")

This Constraint enforces for each UE $u \in \bar{N}_{ue}$ and quality $q \in N_{qul}^{v, s}$ of segment $s \in N_{seg}^{v}$ of the video $v \in N_{vids}^{}$ be a continuous path established between the gNB hosting the UE and the gNB providing  the  requested  file.

\begin{equation}\label{eq:6}
\begin{split}
&{n \in N, \forall e^{u, q} \in \bar{E}:} \\
&  \sum_{e \in E^{n \rightarrow}} \chi^{e^{u, q}}_{e} -
\sum_{e \in E^{\rightarrow n}} \chi^{e^{u, q}}_{e} = \left \{
\begin{aligned}
&-1 && \text{if}\ n = u \\
&1 && \text{if}\ n = q \\
&0 && \text{otherwise} 
\end{aligned} \right.
\end{split}
\end{equation}
where $E^{n \rightarrow}$ represents the links originating from node $n \in N$, while $E^{\rightarrow n}$ represents all the links entering node $n \in N$.

In [75]:
for u in N_ue: 
        for n in N:
            if n in N_gnb:
                if bsAssociation[u] == n:
                    xx = 1
                else:
                    xx = 0
                model.addConstr(-1 * xx + quicksum(X_vsq_nu[userVidSegQaul[u][0],userVidSegQaul[u][1], h, n, u] \
                                    for h in N_qul) \
                   + quicksum(X_ebar_e[u, e] for e in G.edges if e[0] == n ) - \
                     quicksum(X_ebar_e[u, e] for e in G.edges if e[1] == n ) == 0, name ="RR")         
            else:
                 model.addConstr(quicksum(X_vsq_nu[userVidSegQaul[u][0],userVidSegQaul[u][1], h, n, u] \
                                    for h in N_qul) \
                   + quicksum(X_ebar_e[u, e] for e in G.edges if e[0] == n ) - \
                     quicksum(X_ebar_e[u, e] for e in G.edges if e[1] == n ) == 0, name ="RR")         


Video transcoding refers to the process of taking an existing video file (or ongoing stream) and re-encoding it with a different codec or different settings to make it to a lower bit-rate (quality) version. With this constraint we are setting limits over the maximum possible number of videos that can be transcoded from a video segment with quality $h \in N_{qul}^{v, s}$ to a lower bitrate $q \in N_{qul}^{v, s}$ that is requested by the user.
\begin{equation}\label{eq:4}
 {\forall n \in N_{gnb}:\sum_{u \in \bar{N}_{ue}} \sum _{\substack{h \in N_{qul}^{v, s} \\ h > q}} \chi_{n, u}^{v, s, h} - \Upsilon(h, q, n)\leq 0 }
\end{equation}



In [76]:
for n in G.nodes:
    if n in N_gnb:
        model.addConstr(quicksum(X_vsq_nu[userVidSegQaul[u][0],userVidSegQaul[u][1], h, n, u] \
            for u in N_ue for h in N_qul if h > userVidSegQaul[u][2]) - maxTransQual[h, userVidSegQaul[u][2]] <= 0 , name = "C7")

# Model Optimization

In [77]:
#model.Params.IntFeasTol = 1e-9
model.optimize()
model.getVars()

Optimize a model with 106 rows, 140 columns and 290 nonzeros
Variable types: 0 continuous, 140 integer (140 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+07]
  Objective range  [1e+00, 1e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+10]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 10018.000000
Presolve removed 101 rows and 122 columns
Presolve time: 0.00s
Presolved: 5 rows, 18 columns, 32 nonzeros
Variable types: 0 continuous, 18 integer (18 binary)

Root relaxation: objective 2.001800e+04, 1 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    20018.000000 20018.0000  0.00%     -    0s

Explored 0 nodes (1 simplex iterations) in 0.03 seconds
Thread count was 4 (of 4 available processors)

Solutio

[<gurobi.Var X_vsq_n [Vid2, Seg1, Qual2, Node1] (value 0.0)>,
 <gurobi.Var X_vsq_n [Vid2, Seg1, Qual2, Node0] (value 1.0)>,
 <gurobi.Var X_vsq_n [Vid2, Seg1, Qual2, Node3] (value 0.0)>,
 <gurobi.Var X_vsq_n [Vid2, Seg1, Qual2, Node2] (value 0.0)>,
 <gurobi.Var X_vsq_n [Vid2, Seg1, Qual3, Node1] (value 0.0)>,
 <gurobi.Var X_vsq_n [Vid2, Seg1, Qual3, Node0] (value 1.0)>,
 <gurobi.Var X_vsq_n [Vid2, Seg1, Qual3, Node3] (value 0.0)>,
 <gurobi.Var X_vsq_n [Vid2, Seg1, Qual3, Node2] (value 0.0)>,
 <gurobi.Var X_vsq_n [Vid2, Seg1, Qual1, Node1] (value -0.0)>,
 <gurobi.Var X_vsq_n [Vid2, Seg1, Qual1, Node0] (value 1.0)>,
 <gurobi.Var X_vsq_n [Vid2, Seg1, Qual1, Node3] (value -0.0)>,
 <gurobi.Var X_vsq_n [Vid2, Seg1, Qual1, Node2] (value -0.0)>,
 <gurobi.Var X_vsq_n [Vid2, Seg1, Qual4, Node1] (value 0.0)>,
 <gurobi.Var X_vsq_n [Vid2, Seg1, Qual4, Node0] (value 1.0)>,
 <gurobi.Var X_vsq_n [Vid2, Seg1, Qual4, Node3] (value -0.0)>,
 <gurobi.Var X_vsq_n [Vid2, Seg1, Qual4, Node2] (value 0.0)>,
 <gu

## <p style="text-align:center" color='red' ><font color='red'> Performance Evaluation </font> </p>
   


# Execution time

In [78]:
kpifolder = '/executionTime.txt'
filename = '../Results/' + objectiveName + '/executionTime' + kpifolder

f = open(filename, 'a+' )
f.write("Number of Users: ")
f.write(repr(n_ue))
f.write("\n\nExecution Time: ")
f.write(repr(model.Runtime))
f.write("\n****************************\n")
f.close()    


# Computig Cache Hit

In [79]:
kpifolder = '/cacheHit.txt'
filename = '../Results/' + objectiveName + '/cacheHit' + kpifolder 

cacheHit = 0
for u in N_ue:
    for h in N_qul:
        for n in N_gnb:
            if X_vsq_nu[userVidSegQaul[u][0],userVidSegQaul[u][1], h, n, u].X == 1:
                cacheHit +=1 
                
f = open(filename, 'a+' )
f.write("Number of Users: ")
f.write(repr(n_ue))
f.write("\n\nCache Hit at the gNBs: ")
f.write(repr(cacheHit))
f.write("\n****************************\n")
f.close()      

# Computing Byte Hit

In [2]:
# READ: Integrated Prefetching and Caching for Adaptive Video Streaming over HTTP: An Online Approach

In [3]:
kpifolder = '/byteHit.txt'
filename = '../Results/' + objectiveName + '/byteHit' + kpifolder 

byteHit = 0
for u in N_ue:
    for h in N_qul:
        for n in N_gnb:
            if X_vsq_nu[userVidSegQaul[u][0],userVidSegQaul[u][1], h, n, u].X == 1:
                byteHit += bitrate[h] * segDuration 
                
f = open(filename, 'a+' )
f.write("Number of Users: ")
f.write(repr(n_ue))
f.write("\n\nByte Hit at the gNBs: ")
f.write(repr(byteHit))
f.write("\n****************************\n")
f.close()      

NameError: name 'objectiveName' is not defined

# Xn Link Utilization

In [81]:
kpifolder = '/x2LinkUtilization.txt'
filename = '../Results/' + objectiveName + '/x2LinkUtilization' + kpifolder 

x2LinkUtilization = 0
for u in N_ue:
    for e in G.edges:
        if X_ebar_e[u, e].X == 1:
            if e[0] in N_gnb & e[1] in N_gnb:
                x2LinkUtilization += size[v, s, q]
                
f = open(filename, 'a+' )
f.write("Number of Users: ")
f.write(repr(n_ue))
f.write("\n\nx2LinkUtilization: ")
f.write(repr(x2LinkUtilization))
f.write("\n****************************\n")
f.close() 

# Backhaul Link Utilization

kpifolder = '/bhLinkUtilization.txt'
filename = '../Results/' + objectiveName + '/bhLinkUtilization' + kpifolder 

bhLinkUtilization = 0
for u in N_ue:
    for e in G.edges:
        if X_ebar_e[u, e].X == 1:
            if e[0] not in N_gnb & e[1] not in N_gnb:
                bhLinkUtilization += size[v, s, q]
                
f = open(filename, 'a+' )
f.write("Number of Users: ")
f.write(repr(n_ue))
f.write("\n\nbhLinkUtilization: ")
f.write(repr(bhLinkUtilization))
f.write("\n****************************\n")
f.close() 

# Video Quality and Number of Users

Here we compute the number of users that are using a segment quality ex. how many users are using video segment with Q1, Q2, and so on

# Number of video segments over all the edge nodes

This metric can evaluate the performance of the two algorithms to see their trend in caching videos. We expect that the first objective caches more video segments

# Average Bitrate for each user

In [83]:
# https://www.youtube.com/watch?v=VSqaw9pyGis


# Access Latency

The objective of prefetching is to maximize the byte-hit ratio by predicting future requests. To do this, prefetching
will fetch uncached data before users actually need it. As a result, the access latency is reduced when the user requests
for the prefetched data arrive at proxies, resulting in better user perceived quality. However, prefetching aggressively is
a waste of network resources, since the prefetched data may not be actually used as video players are free to switch between different bitrates in HTTP-based adaptive streaming
applications.

In [84]:
print 'DONE!'

DONE!
